# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 59 new papers today
          25 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/24 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.02056


extracting tarball to tmp_2211.02056...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.02060


extracting tarball to tmp_2211.02060...

...

extracting tarball to tmp_2211.02547...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.02565


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.02547/TOI4524_KESPRINT_Murgas.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'table-TOI-4524-tng_harpn-0120-srv-complete_output' from 'tmp_2211.02547/table-TOI-4524-tng_harpn-0120-srv-complete_output.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'table-TOI-4524-tng_harpn-0120-drs-complete_output.tex' from 'tmp_2211.02547/table-TOI-4524-tng_harpn-0120

extracting tarball to tmp_2211.02565...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.02502-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.02502) | **Salt-bearing disk candidates around high-mass young stellar objects**  |
|| Adam Ginsburg, et al. -- incl., <mark>Yichen Zhang</mark>, <mark>Henrik Beuther</mark> |
|*Appeared on*| *2022-11-07*|
|*Comments*| *accepted to ApJ*|
|**Abstract**| Molecular lines tracing the orbital motion of gas in a well-defined disk are valuable tools for inferring both the properties of the disk and the star it surrounds. Lines that arise only from a disk, and not also from the surrounding molecular cloud core that birthed the star or from the outflow it drives, are rare. Several such emission lines have recently been discovered in one example case, those from NaCl and KCl salt molecules. We studied a sample of 23 candidate high-mass young stellar objects (HMYSOs) in 17 high-mass star-forming regions to determine how frequently emission from these species is detected. We present five new detections of water, NaCl, KCl, PN, and SiS from the innermost regions around the objects, bringing the total number of known briny disk candidates to nine. Their kinematic structure is generally disk-like, though we are unable to determine whether they arise from a disk or outflow in the sources with new detections. We demonstrate that these species are spatially coincident in a few resolved cases and show that they are generally detected together, suggesting a common origin or excitation mechanism. We also show that several disks around HMYSOs clearly do not exhibit emission in these species. Salty disks are therefore neither particularly rare in high-mass disks, nor are they ubiquitous. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.02056-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.02056) | **A spatially resolved analysis of star-formation burstiness by comparing  UV and H$α$ in galaxies at z$\sim$1 with UVCANDELS**  |
|| Vihang Mehta, et al. -- incl., <mark>Yicheng Guo</mark> |
|*Appeared on*| *2022-11-07*|
|*Comments*| *19 pages, 8 figures; submitted to ApJ*|
|**Abstract**| The UltraViolet imaging of the Cosmic Assembly Near-infrared Deep Extragalactic Legacy Survey Fields (UVCANDELS) program provides HST/UVIS F275W imaging for four CANDELS fields. We combine this UV imaging with existing HST/near-IR grism spectroscopy from 3D-HST+AGHAST to directly compare the resolved rest-frame UV and H$\alpha$ emission for a sample of 979 galaxies at $0.7<z<1.5$ spanning a range in stellar mass of $10^{8-11.5}~M_\odot$. Since both rest-UV and H$\alpha$ are sensitive to on-going star-formation but over different timescales, their resolved comparison allows us to infer the burstiness in star-formation as a function of galaxy structural parameters. We generate homogenized maps of rest-UV and H$\alpha$ emission for all galaxies in our sample and stack them to compute the average UV-to-H$\alpha$ luminosity ratio as a function of galactocentric radius. We find that galaxies below stellar mass of $\sim10^{9.5}~M_\odot$, at all radii, have a UV-to-H$\alpha$ ratio higher than the equilibrium value expected from constant star-formation, indicating a significant contribution from bursty star-formation. Even for galaxies with stellar mass $\gtrsim10^{9.5} M_\odot$, the UV-to-H$\alpha$ ratio is elevated towards in their outskirts ($R/R_{eff}>1.5$), suggesting that bursty star-formation is likely prevalent in the outskirts of even the most massive galaxies but is likely over-shadowed by their brighter cores. Furthermore, we present the UV-to-H$\alpha$ ratio as a function of galaxy surface brightness, a proxy for stellar mass surface density, and find that regions below $\sim10^8~M_\odot~kpc^{-2}$ are consistent with bursty star-formation, regardless of their galaxy stellar mass, potentially suggesting that local star-formation is independent of global galaxy properties at the smallest scales. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.02060-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.02060) | **On the patchy appearance of the circum-Galactic medium and the influence  of foreground absorption**  |
|| G. Ponti, et al. -- incl., <mark>Y. Zhang</mark> |
|*Appeared on*| *2022-11-07*|
|*Comments*| *Accepted for publication in Astronomy and Astrophysics*|
|**Abstract**| Recent studies have demonstrated that the emission from the circumgalactic medium of the Milky Way displays a relatively high degree of patchiness on angular scales of $\sim10^\circ$. Taking advantage of the Spectrum Roentgen Gamma eROSITA Final Equatorial Depth Survey (eFEDS), we aim to constrain any variation in the soft X-ray surface brightness on scales going from sub-degree to a hundred square degrees. We observe modulations of about $60$% on scales of several degrees and decreasing for higher energies. The observed patchiness is stable over a period of two years, therefore excluding that it is induced by Solar wind charge exchange. We also observe no correlation between such excess and the density of galaxies in the Local Universe, suggesting no strong contribution from the hot baryons in the filaments of the Cosmic web. Instead, the soft X-ray emission is anti-correlated with the column density of absorbing material. Indeed, we can reproduce the spectrum of the bright and dark regions by simply varying the column density of the matter absorbing the emission components located beyond the Local Hot Bubble, while no modulation of the intrinsic emission is required. At high Galactic latitudes, the eROSITA all sky map shows patchiness of the soft X-ray diffuse emission similar to the one observed in the eFEDS field, it is therefore likely that the same "absorption-modulation" is present over the entire sky. These results highlight the importance of an accurate treatment of the absorption effects, to determine the patchiness of the circumgalactic medium. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.02526-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.02526) | **Constraints on neutron star superfluidity from the cooling neutron star  in Cassiopeia A using all Chandra ACIS-S observations**  |
|| Peter S. Shternin, Dmitry D. Ofengeim, <mark>Craig O. Heinke</mark>, Wynn C.G. Ho |
|*Appeared on*| *2022-11-07*|
|*Comments*| *20 pages, 17 figures. Accepted for publication in MNRAS*|
|**Abstract**| Analysis of Chandra observations of the neutron star (NS) in the centre of the Cassiopeia A supernova remnant taken in the subarray (FAINT) mode of the ACIS detector performed by Posselt and collaborators revealed, after inclusion of the most recent (May 2020) observations, a significant decrease of the source surface temperature from 2006 to 2020. The obtained cooling rate is consistent with those obtained from analysis of the 2000$-$2019 data taken in the GRADED mode of the ACIS detector, which is potentially more strongly affected by instrumental effects. We performed a joint spectral analysis using all ACIS data to constrain the NS parameters and cooling rate. We constrain the mass of the Cassiopeia A NS at $M=1.55\pm0.25~M_\odot$, and its radius at $R=13.5\pm 1.5$ km. The surface temperature cooling rate is found to be $2.2\pm 0.3$ per cent in 10 years if the absorbing hydrogen column density is allowed to vary and $1.6\pm 0.2$ per cent in 10 years if it is fixed. The observed cooling can be explained by enhanced neutrino emission from the superfluid NS interior due to Cooper Pair Formation (CPF) process. Based on analysis of all ACIS data, we constrain the maximal critical temperature of triplet neutron pairing within the NS core at $(4-9.5)\times 10^{8}$ K. In accordance with previous studies, the required effective strength of the CPF neutrino emission is at least a factor of 2 higher than existing microscopic calculations suggest. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.02547-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.02547) | **HD 20329b: An ultra-short-period planet around a solar-type star found  by TESS**  |
|| F. Murgas, et al. -- incl., <mark>John Livingston</mark>, <mark>Alexis M. S. Smith</mark>, <mark>Carl Ziegler</mark> |
|*Appeared on*| *2022-11-07*|
|*Comments*| *Accepted for publication in A&A, 26 pages*|
|**Abstract**| We used TESS light curves and HARPS-N spectrograph radial velocity measurements to establish the physical properties of the transiting exoplanet candidate found around the star HD 20329 (TOI-4524). We performed a joint fit of the light curves and radial velocity time series to measure the mass, radius, and orbital parameters of the candidate. We confirm and characterize HD 20329b, an ultra-short-period (USP) planet transiting a solar-type star. The host star (HD 20329, $V = 8.74$ mag, $J = 7.5$ mag) is characterized by its G5 spectral type with $\mathrm{M}_\star= 0.90 \pm 0.05$ M$_\odot$, $\mathrm{R}_\star = 1.13 \pm 0.02$ R$_\odot$, and $\mathrm{T}_{\mathrm{eff}} = 5596 \pm 50$ K; it is located at a distance $d= 63.68 \pm 0.29$ pc. By jointly fitting the available TESS transit light curves and follow-up radial velocity measurements, we find an orbital period of $0.9261 \pm (0.5\times 10^{-4})$ days, a planetary radius of $1.72 \pm 0.07$ $\mathrm{R}_\oplus$, and a mass of $7.42 \pm 1.09$ $\mathrm{M}_\oplus$, implying a mean density of $\rho_\mathrm{p} = 8.06 \pm 1.53$ g cm$^{-3}$. HD 20329b joins the $\sim$30 currently known USP planets with radius and Doppler mass measurements. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.02565-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.02565) | **Coherent and incoherent structures in fuzzy dark matter halos**  |
|| <mark>I-Kang Liu</mark>, Nick P. Proukakis, Gerasimos Rigopoulos |
|*Appeared on*| *2022-11-07*|
|*Comments*| **|
|**Abstract**| We show that fuzzy dark matter halos exhibit spatial differentiation in the degree of coherence of the field configuration, ranging from completely coherent in the central solitonic core to incoherent outside it, with a crossover region in between the two phases. The solitonic core is indeed a pure condensate which overlaps almost perfectly with the Penrose-Onsager mode corresponding to the largest eigenvalue of the one-particle density matrix. The virialized outer halo surrounding the core exhibits no clear coherence as a whole upon radial and temporal averaging. However, when viewed locally and for short times, it can be described as a collection of quasi-condensate lumps exhibiting locally suppressed fluctuations which can be identified with the structures commonly referred to as granules. Phase coherence across the entire halo is inhibited by a dynamically evolving tangled web of vortices separating the localized quasi-condensate regions. Moreover, the dimensionless phase-space density in the outer halo drops significantly below its value at the core. We further examine the dynamics of this spatial structure and find that the oscillations of the core can be accurately described by two time-dependent parameters respectively characterizing the size of the core, $r_c(t)$, and the crossover region, $r_t(t)$. For the halos produced in our merger simulations this feature is reflected in the (anti-)correlated oscillation of the peak value of the field configuration's power-spectrum. The turbulent vortex tangle of the virialized halo appears to reach a quasi-equilibrium state over probed timescales, with the incompressible component of the kinetic energy exhibiting a characteristic $k^{-3}$ tail in its spectrum, indicative of a $\rho\sim r^2$ density profile around the quantum vortex cores. Comparison of the peak wavenumbers in the corresponding power-spectra shows that the inter-vortex... |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.02137-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.02137) | **Photometric and spectroscopic study of the EXor-like eruptive young star  Gaia19fct**  |
|| Sunkyung Park, et al. -- incl., <mark>Jeong-Eun Lee</mark>, <mark>Jae-Joon Lee</mark> |
|*Appeared on*| *2022-11-07*|
|*Comments*| *29 pages, 14 figures, 7 tables, Accepted for publication in ApJ*|
|**Abstract**| Gaia19fct is one of the Gaia-alerted eruptive young stars that has undergone several brightening events. We conducted monitoring observations using multi-filter optical and near-infrared photometry, as well as near-infrared spectroscopy, to understand the physical properties of Gaia19fct and investigate whether it fits into the historically defined two classes. We present the analyses of light curves, color variations, spectral lines, and CO modeling. The light curves show at least five brightening events since 2015, and the multi-filter color evolutions are mostly gray. The gray evolution indicates that bursts are triggered by mechanisms other than extinction. Our near-infrared spectra exhibit both absorption and emission lines and show time-variability throughout our observations. We found lower rotational velocity and lower temperature from the near-infrared atomic absorption lines than from the optical lines, suggesting that Gaia19fct has a Keplerian rotating disk. The CO overtone features show a superposition of absorption and emission components, which is unlike other young stellar objects. We modeled the CO lines, and the result suggests that the emission and absorption components are formed in different regions. We found that although Gaia19fct exhibits characteristics of both types of eruptive young stars, FU Orionis-type objects (FUors) and EX Lupi-type objects (EXors), it shows more similarity with EXors in general. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.02428-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.02428) | **Fractal signature as a rotational modulation and stellar noise  classifier based on the active Kepler stars**  |
|| Paulo Cleber Farias da Silva Filho, et al. -- incl., <mark>Fernando Jose Silva Lima Filho</mark> |
|*Appeared on*| *2022-11-07*|
|*Comments*| *18 pages, 5 figures, submitted to BJD*|
|**Abstract**| In this study, we report on the analysis of 701 stars in a solar vicinity defined in three categories namely subsolar, solar, and supersolar with rotation periods between 1 and 70 days, based on rotational modulation signatures inferred from time series from the Kepler mission's Public Archives. In our analysis, we performed an initial selection based on the rotation period and position in the period-H diagram, where H denotes the Hurst exponent extracted from fractal analysis. To refine our analysis, we applied a fractal approach known as the R/S method, taking into account the fluctuations of the features associated with photometric modulation at different time intervals and the fractality traces that are present in the time series of our sample. In this sense, we computed the so-called Hurst exponent for the referred stars and found that it can provide a strong discriminant of rotational modulation and background noise behavior, going beyond what can be achieved with solely the rotation period itself. Furthermore, our results emphasize that the rotation period of stars is scaled by the exponent H which increases following the increase in the rotation period. Finally, our approach suggests that the referred exponent may be a powerful rotational modulation and noise classifier. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.02502.md
    + _build/html/tmp_2211.02502/./f1.png
    + _build/html/tmp_2211.02502/./f2.png
    + _build/html/tmp_2211.02502/./f3.png
    + _build/html/tmp_2211.02502/./f4.png
    + _build/html/tmp_2211.02502/./f29.png
    + _build/html/tmp_2211.02502/./f30.png
    + _build/html/tmp_2211.02502/./f31.png
    + _build/html/tmp_2211.02502/./f32.png
    + _build/html/tmp_2211.02502/./f24.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\url}[1]{\href{#1}{#1}}$
$\newcommand{\dodoi}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{\doeprint}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{\doarXiv}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\paa}{Pa\ensuremath{\alpha}}$
$\newcommand{\brg}{Br\ensuremath{\gamma}}$
$\newcommand{\msun}{\ensuremath{M_{\odot}}\xspace}$
$\newcommand{\mdot}{\ensuremath{\dot{M}}\xspace}$
$\newcommand{\lsun}{\ensuremath{L_{\odot}}\xspace}$
$\newcommand{\rsun}{\ensuremath{R_{\odot}}\xspace}$
$\newcommand{\lbol}{\ensuremath{L_{\mathrm{bol}}\xspace}}$
$\newcommand{\ks}{K\ensuremath{_{\mathrm{s}}}}$
$\newcommand{\hh}{\ensuremath{\textrm{H}_{2}}\xspace}$
$\newcommand{\dens}{\ensuremath{n(\hh) [\percc]}\xspace}$
$\newcommand{\formaldehyde}{\ensuremath{\textrm{H}_2\textrm{CO}}\xspace}$
$\newcommand{\formamide}{\ensuremath{\textrm{NH}_2\textrm{CHO}}\xspace}$
$\newcommand{\formaldehydeIso}{\ensuremath{\textrm{H}_2~^{13}\textrm{CO}}\xspace}$
$\newcommand{\methanol}{\ensuremath{\textrm{CH}_3\textrm{OH}}\xspace}$
$\newcommand{\ortho}{\ensuremath{\textrm{o-H}_2\textrm{CO}}\xspace}$
$\newcommand{\para}{\ensuremath{\textrm{p-H}_2\textrm{CO}}\xspace}$
$\newcommand{\oneone}{\ensuremath{1_{1,0}-1_{1,1}}\xspace}$
$\newcommand{\twotwo}{\ensuremath{2_{1,1}-2_{1,2}}\xspace}$
$\newcommand{\threethree}{\ensuremath{3_{1,2}-3_{1,3}}\xspace}$
$\newcommand{\threeohthree}{\ensuremath{3_{0,3}-2_{0,2}}\xspace}$
$\newcommand{\threetwotwo}{\ensuremath{3_{2,2}-2_{2,1}}\xspace}$
$\newcommand{\threetwoone}{\ensuremath{3_{2,1}-2_{2,0}}\xspace}$
$\newcommand{\fourtwotwo}{\ensuremath{4_{2,2}-3_{1,2}}\xspace}$
$\newcommand{\methylcyanide}{\ensuremath{\textrm{CH}_{3}\textrm{CN}}\xspace}$
$\newcommand{\ketene}{\ensuremath{\textrm{H}_{2}\textrm{CCO}}\xspace}$
$\newcommand{\ethylcyanide}{\ensuremath{\textrm{CH}_3\textrm{CH}_2\textrm{CN}}\xspace}$
$\newcommand{\cyanoacetylene}{\ensuremath{\textrm{HC}_{3}\textrm{N}}\xspace}$
$\newcommand{\methylformate}{\ensuremath{\textrm{CH}_{3}\textrm{OCHO}}\xspace}$
$\newcommand{\dimethylether}{\ensuremath{\textrm{CH}_{3}\textrm{OCH}_{3}}\xspace}$
$\newcommand{\gaucheethanol}{\ensuremath{\textrm{g-CH}_3\textrm{CH}_2\textrm{OH}}\xspace}$
$\newcommand{\acetone}{\ensuremath{\left[\textrm{CH}_{3}\right]_2\textrm{CO}}\xspace}$
$\newcommand{\methyleneamidogen}{\ensuremath{\textrm{H}_{2}\textrm{CN}}\xspace}$
$\newcommand{\Rone}{\ensuremath{\para~S_{\nu}(\threetwoone) / S_{\nu}(\threeohthree)}\xspace}$
$\newcommand{\Rtwo}{\ensuremath{\para~S_{\nu}(\threetwotwo) / S_{\nu}(\threetwoone)}\xspace}$
$\newcommand{\JKaKc}{\ensuremath{J_{K_a K_c}}}$
$\newcommand{\water}{H_{2}O\xspace}$
$\newcommand{\feii}{\ion{Fe}{ii}\xspace}$
$\newcommand{\uchii}{\ion{UCH}{ii}\xspace}$
$\newcommand{\UCHII}{\ion{UCH}{ii}\xspace}$
$\newcommand{\hchii}{\ion{HCH}{ii}\xspace}$
$\newcommand{\HCHII}{\ion{HCH}{ii}\xspace}$
$\newcommand{\hii}{\ion{H}{ii}\xspace}$
$\newcommand{\hi}{H~{\sc i}\xspace}$
$\newcommand{\Hii}{\hii}$
$\newcommand{\HII}{\hii}$
$\newcommand{\Xform}{\ensuremath{X_{\formaldehyde}}}$
$\newcommand{\kms}{\textrm{km~s}\ensuremath{^{-1}}\xspace}$
$\newcommand{\nsample}{456\xspace}$
$\newcommand{\CFR}{5\xspace}$
$\newcommand{\permyr}{\ensuremath{\mathrm{Myr}^{-1}}\xspace}$
$\newcommand{\pers}{\ensuremath{\mathrm{s}^{-1}}\xspace}$
$\newcommand{\perspc}{\ensuremath{\mathrm{pc}^{-2}}\xspace}$
$\newcommand{\tsuplim}{0.5\xspace}$
$\newcommand{\ncandidates}{18\xspace}$
$\newcommand{\mindist}{8.7\xspace}$
$\newcommand{\rcluster}{2.5\xspace}$
$\newcommand{\ncomplete}{13\xspace}$
$\newcommand{\middistcut}{13.0\xspace}$
$\newcommand{\nMPC}{3\xspace}$
$\newcommand{\obsfrac}{30}$
$\newcommand{\nMPCtot}{10\xspace}$
$\newcommand{\nMPCtoterr}{6\xspace}$
$\newcommand{\plaw}{2.1\xspace}$
$\newcommand{\plawerr}{0.3\xspace}$
$\newcommand{\mmin}{\ensuremath{10^4~\msun}\xspace}$
$\newcommand{\kmspc}{\textrm{km~s}\ensuremath{^{-1}}\textrm{pc}\ensuremath{^{-1}}\xspace}$
$\newcommand{\sqcm}{cm^{2}\xspace}$
$\newcommand{\percc}{\ensuremath{\textrm{cm}^{-3}}\xspace}$
$\newcommand{\perpc}{\ensuremath{\textrm{pc}^{-1}}\xspace}$
$\newcommand{\persc}{\ensuremath{\textrm{cm}^{-2}}\xspace}$
$\newcommand{\persr}{\ensuremath{\textrm{sr}^{-1}}\xspace}$
$\newcommand{\peryr}{\ensuremath{\textrm{yr}^{-1}}\xspace}$
$\newcommand{\perkmspc}{\textrm{km~s}\ensuremath{^{-1}}\textrm{pc}\ensuremath{^{-1}}\xspace}$
$\newcommand{\perkms}{\textrm{per~km~s}\ensuremath{^{-1}}\xspace}$
$\newcommand{\um}{\ensuremath{\mu \textrm{m}}\xspace}$
$\newcommand{\microjy}{\ensuremath{\mu\textrm{Jy}}\xspace}$
$\newcommand{\microJy}{\ensuremath{\mu\textrm{Jy}}\xspace}$
$\newcommand{\mum}{\um}$
$\newcommand{\htwo}{\ensuremath{\textrm{H}_2}}$
$\newcommand{\Htwo}{\ensuremath{\textrm{H}_2}}$
$\newcommand{\HtwoO}{\ensuremath{\textrm{H}_2\textrm{O}}}$
$\newcommand{\htwoo}{\ensuremath{\textrm{H}_2\textrm{O}}}$
$\newcommand{\ha}{\ensuremath{\textrm{H}\alpha}}$
$\newcommand{\hb}{\ensuremath{\textrm{H}\beta}}$
$\newcommand{\so}{SO~\ensuremath{5_6-4_5}\xspace}$
$\newcommand{\SO}{SO~\ensuremath{1_2-1_1}\xspace}$
$\newcommand{\ammonia}{NH\ensuremath{_3}\xspace}$
$\newcommand{\twelveco}{\ensuremath{^{12}\textrm{CO}}\xspace}$
$\newcommand{\thirteenco}{\ensuremath{^{13}\textrm{CO}}\xspace}$
$\newcommand{\ceighteeno}{\ensuremath{\textrm{C}^{18}\textrm{O}}\xspace}$
$\newcommand{\degrees}{\ensuremath{^{\circ}}}$
$\newcommand{\lowirac}{800}$
$\newcommand{\highirac}{8000}$
$\newcommand{\lowmips}{600}$
$\newcommand{\highmips}{5000}$
$\newcommand{\perbeam}{\ensuremath{\textrm{beam}^{-1}}\xspace}$
$\newcommand{\ds}{\ensuremath{\textrm{d}s}}$
$\newcommand{\dnu}{\ensuremath{\textrm{d}\nu}}$
$\newcommand{\dv}{\ensuremath{\textrm{d}v}}$
$\newcommand{\necluster}{Sh~2-233IR~NE}$
$\newcommand{\swcluster}{Sh~2-233IR~SW}$
$\newcommand{\region}{IRAS 05358}$
$\newcommand{\nwfive}{40}$
$\newcommand{\nouter}{15}$
$\newcommand{\vone}{{\rm v}1.0\xspace}$
$\newcommand{\vtwo}{{\rm v}2.0\xspace}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\todo}[1]{\textcolor{red}{#1}}$
$\newcommand{\okinfinal}[1]{{#1}}$
$\newcommand{\ndisks}{9\xspace}$
$\newcommand{\ntentative}{3\xspace}$
$\newcommand{\ndiskstext}{nine\xspace}$
$\newcommand{\nnewdisks}{5\xspace}$
$\newcommand{\nregions}{6\xspace}$
$\newcommand{\}{ee}$
$\newcommand{\}{secref}$
$\newcommand{\}{eqref}$
$\newcommand{\}{facility}$
$\newcommand{\}{natexlab}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\url$}[1]{\href{#1}{#1}}$
$\newcommand{$\dodoi$}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{$\doeprint$}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{$\doarXiv$}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\paa$}{Pa$\ensuremath${\alpha}}$
$\newcommand{$\brg$}{Br$\ensuremath${\gamma}}$
$\newcommand{$\msun$}{$\ensuremath${M_{\odot}}$\xspace$}$
$\newcommand{$\mdot$}{$\ensuremath${\dot{M}}$\xspace$}$
$\newcommand{$\lsun$}{$\ensuremath${L_{\odot}}$\xspace$}$
$\newcommand{$\rsun$}{$\ensuremath${R_{\odot}}$\xspace$}$
$\newcommand{$\lbol$}{$\ensuremath${L_{\mathrm{bol}}$\xspace$}}$
$\newcommand{$\ks$}{K$\ensuremath${_{\mathrm{s}}}}$
$\newcommand{$\hh$}{$\ensuremath${\textrm{H}_{2}}$\xspace$}$
$\newcommand{$\dens$}{$\ensuremath${n($\hh$) [$\percc$]}$\xspace$}$
$\newcommand{$\formaldehyde$}{$\ensuremath${\textrm{H}_2\textrm{CO}}$\xspace$}$
$\newcommand{$\formamide$}{$\ensuremath${\textrm{NH}_2\textrm{CHO}}$\xspace$}$
$\newcommand{$\formaldehyde$Iso}{$\ensuremath${\textrm{H}_2~^{13}\textrm{CO}}$\xspace$}$
$\newcommand{$\methanol$}{$\ensuremath${\textrm{CH}_3\textrm{OH}}$\xspace$}$
$\newcommand{$\ortho$}{$\ensuremath${\textrm{o-H}_2\textrm{CO}}$\xspace$}$
$\newcommand{$\para$}{$\ensuremath${\textrm{p-H}_2\textrm{CO}}$\xspace$}$
$\newcommand{$\oneone$}{$\ensuremath${1_{1,0}-1_{1,1}}$\xspace$}$
$\newcommand{$\twotwo$}{$\ensuremath${2_{1,1}-2_{1,2}}$\xspace$}$
$\newcommand{$\threethree$}{$\ensuremath${3_{1,2}-3_{1,3}}$\xspace$}$
$\newcommand{$\threeohthree$}{$\ensuremath${3_{0,3}-2_{0,2}}$\xspace$}$
$\newcommand{$\threetwotwo$}{$\ensuremath${3_{2,2}-2_{2,1}}$\xspace$}$
$\newcommand{$\threetwoone$}{$\ensuremath${3_{2,1}-2_{2,0}}$\xspace$}$
$\newcommand{$\fourtwotwo$}{$\ensuremath${4_{2,2}-3_{1,2}}$\xspace$}$
$\newcommand{$\methylcyanide$}{$\ensuremath${\textrm{CH}_{3}\textrm{CN}}$\xspace$}$
$\newcommand{$\ketene$}{$\ensuremath${\textrm{H}_{2}\textrm{CCO}}$\xspace$}$
$\newcommand{$\ethylcyanide$}{$\ensuremath${\textrm{CH}_3\textrm{CH}_2\textrm{CN}}$\xspace$}$
$\newcommand{$\cyanoacetylene$}{$\ensuremath${\textrm{HC}_{3}\textrm{N}}$\xspace$}$
$\newcommand{$\methylformate$}{$\ensuremath${\textrm{CH}_{3}\textrm{OCHO}}$\xspace$}$
$\newcommand{$\dimethylether$}{$\ensuremath${\textrm{CH}_{3}\textrm{OCH}_{3}}$\xspace$}$
$\newcommand{$\gaucheethanol$}{$\ensuremath${\textrm{g-CH}_3\textrm{CH}_2\textrm{OH}}$\xspace$}$
$\newcommand{$\acetone$}{$\ensuremath${\left[\textrm{CH}_{3}\right]_2\textrm{CO}}$\xspace$}$
$\newcommand{$\methyleneamidogen$}{$\ensuremath${\textrm{H}_{2}\textrm{CN}}$\xspace$}$
$\newcommand{$\Rone$}{$\ensuremath${$\para$~S_{\nu}($\threetwoone$) / S_{\nu}($\threeohthree$)}$\xspace$}$
$\newcommand{$\Rtwo$}{$\ensuremath${$\para$~S_{\nu}($\threetwotwo$) / S_{\nu}($\threetwoone$)}$\xspace$}$
$\newcommand{$\JKaKc$}{$\ensuremath${J_{K_a K_c}}}$
$\newcommand{$\water$}{H_{2}O$\xspace$}$
$\newcommand{$\feii$}{$\ion${Fe}{ii}$\xspace$}$
$\newcommand{$\uchii$}{$\ion${UCH}{ii}$\xspace$}$
$\newcommand{$\UCHII$}{$\ion${UCH}{ii}$\xspace$}$
$\newcommand{$\hchii$}{$\ion${HCH}{ii}$\xspace$}$
$\newcommand{$\HCHII$}{$\ion${HCH}{ii}$\xspace$}$
$\newcommand{$\hii$}{$\ion${H}{ii}$\xspace$}$
$\newcommand{$\hi$}{H~{\sc i}$\xspace$}$
$\newcommand{$\Hii$}{$\hii$}$
$\newcommand{$\HII$}{$\hii$}$
$\newcommand{$\Xform$}{$\ensuremath${X_{$\formaldehyde$}}}$
$\newcommand{$\kms$}{\textrm{km~s}$\ensuremath${^{-1}}$\xspace$}$
$\newcommand{$\nsample$}{456$\xspace$}$
$\newcommand{$\CFR$}{5$\xspace$}$
$\newcommand{$\permyr$}{$\ensuremath${\mathrm{Myr}^{-1}}$\xspace$}$
$\newcommand{$\pers$}{$\ensuremath${\mathrm{s}^{-1}}$\xspace$}$
$\newcommand{$\pers$pc}{$\ensuremath${\mathrm{pc}^{-2}}$\xspace$}$
$\newcommand{$\tsuplim$}{0.5$\xspace$}$
$\newcommand{$\ncandidates$}{18$\xspace$}$
$\newcommand{$\mindist$}{8.7$\xspace$}$
$\newcommand{$\rcluster$}{2.5$\xspace$}$
$\newcommand{$\ncomplete$}{13$\xspace$}$
$\newcommand{$\middistcut$}{13.0$\xspace$}$
$\newcommand{$\nMPC$}{3$\xspace$}$
$\newcommand{$\obsfrac$}{30}$
$\newcommand{$\nMPC$tot}{10$\xspace$}$
$\newcommand{$\nMPC$toterr}{6$\xspace$}$
$\newcommand{$\plaw$}{2.1$\xspace$}$
$\newcommand{$\plaw$err}{0.3$\xspace$}$
$\newcommand{$\mmin$}{$\ensuremath${10^4~$\msun$}$\xspace$}$
$\newcommand{$\kms$pc}{\textrm{km~s}$\ensuremath${^{-1}}\textrm{pc}$\ensuremath${^{-1}}$\xspace$}$
$\newcommand{$\sqcm$}{cm^{2}$\xspace$}$
$\newcommand{$\percc$}{$\ensuremath${\textrm{cm}^{-3}}$\xspace$}$
$\newcommand{$\perpc$}{$\ensuremath${\textrm{pc}^{-1}}$\xspace$}$
$\newcommand{$\pers$c}{$\ensuremath${\textrm{cm}^{-2}}$\xspace$}$
$\newcommand{$\pers$r}{$\ensuremath${\textrm{sr}^{-1}}$\xspace$}$
$\newcommand{$\peryr$}{$\ensuremath${\textrm{yr}^{-1}}$\xspace$}$
$\newcommand{$\perkmspc$}{\textrm{km~s}$\ensuremath${^{-1}}\textrm{pc}$\ensuremath${^{-1}}$\xspace$}$
$\newcommand{$\perkms$}{\textrm{per~km~s}$\ensuremath${^{-1}}$\xspace$}$
$\newcommand{$\um$}{$\ensuremath${\mu \textrm{m}}$\xspace$}$
$\newcommand{$\microjy$}{$\ensuremath${\mu\textrm{Jy}}$\xspace$}$
$\newcommand{$\microJy$}{$\ensuremath${\mu\textrm{Jy}}$\xspace$}$
$\newcommand{$\mum$}{$\um$}$
$\newcommand{$\htwo$}{$\ensuremath${\textrm{H}_2}}$
$\newcommand{$\Htwo$}{$\ensuremath${\textrm{H}_2}}$
$\newcommand{$\Htwo$O}{$\ensuremath${\textrm{H}_2\textrm{O}}}$
$\newcommand{$\htwo$o}{$\ensuremath${\textrm{H}_2\textrm{O}}}$
$\newcommand{$\ha$}{$\ensuremath${\textrm{H}\alpha}}$
$\newcommand{$\hb$}{$\ensuremath${\textrm{H}\beta}}$
$\newcommand{$\so$}{SO~$\ensuremath${5_6-4_5}$\xspace$}$
$\newcommand{$\SO$}{SO~$\ensuremath${1_2-1_1}$\xspace$}$
$\newcommand{$\ammonia$}{NH$\ensuremath${_3}$\xspace$}$
$\newcommand{$\twelveco$}{$\ensuremath${^{12}\textrm{CO}}$\xspace$}$
$\newcommand{$\thirteenco$}{$\ensuremath${^{13}\textrm{CO}}$\xspace$}$
$\newcommand{$\ceighteeno$}{$\ensuremath${\textrm{C}^{18}\textrm{O}}$\xspace$}$
$\newcommand{$\degrees$}{$\ensuremath${^{\circ}}}$
$\newcommand{$\lowirac$}{800}$
$\newcommand{$\hi$ghirac}{8000}$
$\newcommand{$\lowmips$}{600}$
$\newcommand{$\hi$ghmips}{5000}$
$\newcommand{$\perbeam$}{$\ensuremath${\textrm{beam}^{-1}}$\xspace$}$
$\newcommand{$\ds$}{$\ensuremath${\textrm{d}s}}$
$\newcommand{$\dnu$}{$\ensuremath${\textrm{d}\nu}}$
$\newcommand{$\dv$}{$\ensuremath${\textrm{d}v}}$
$\newcommand{$\necluster$}{Sh~2-233IR~NE}$
$\newcommand{$\swcluster$}{Sh~2-233IR~SW}$
$\newcommand{$\region$}{IRAS 05358}$
$\newcommand{$\nwfive$}{40}$
$\newcommand{$\nouter$}{15}$
$\newcommand{$\vone$}{{\rm v}1.0$\xspace$}$
$\newcommand{$\vtwo$}{{\rm v}2.0$\xspace$}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{$\todo$}[1]{\textcolor{red}{#1}}$
$\newcommand{$\okinfinal$}[1]{{#1}}$
$\newcommand{$\ndisks$}{9$\xspace$}$
$\newcommand{$\ntentative$}{3$\xspace$}$
$\newcommand{$\ndisks$text}{nine$\xspace$}$
$\newcommand{$\nnewdisks$}{5$\xspace$}$
$\newcommand{$\nregions$}{6$\xspace$}$
$\newcommand{\}{ee}$
$\newcommand{\}{secref}$
$\newcommand{\}{eqref}$
$\newcommand{\}{facility}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# Salt-bearing disk candidates around high-mass young stellar objects

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.02502-b31b1b.svg)](https://arxiv.org/abs/2211.02502)<mark>Appeared on: 2022-11-07</mark> - _accepted to ApJ_

</div>
<div id="authors">

Adam Ginsburg, et al. -- incl., <mark>Yichen Zhang</mark>, <mark>Henrik Beuther</mark>

</div>
<div id="abstract">

**Abstract:** Molecular lines tracing the orbital motion of gas in a well-defined disk are valuable tools for inferring both the properties of the disk and the star it surrounds.Lines that arise only from a disk, and not also from the surrounding molecular cloud core that birthed the star or from the outflow it drives, are rare.Several such emission lines have recently been discovered in one example case, those from NaCl and KCl salt molecules.We studied a sample of 23 candidate high-mass young stellar objects (HMYSOs) in 17 high-mass star-forming regions to determine how frequently emission from these species is detected.We present$\nnewdisks$new detections of water, NaCl, KCl, PN, and SiS from the innermost regions around the objects, bringing the total number of known briny disk candidates to$\ndisks$.Their kinematic structure is generally disk-like,{though we are unable to determine whether they arise from a disk or outflow in the  sources with new detections}.We demonstrate that these species are spatially coincident in a few resolved cases and show that they are generally detected together, suggesting a common origin or excitation mechanism.We also show that several disks around HMYSOs clearly do not exhibit emission in these species.Salty disks are therefore neither particularly rare in high-mass disks, nor are they ubiquitous.

</div>

<div id="div_fig1">

<img src="tmp_2211.02502/./f1.png" alt="Fig5.1" width="25%"/><img src="tmp_2211.02502/./f2.png" alt="Fig5.2" width="25%"/><img src="tmp_2211.02502/./f3.png" alt="Fig5.3" width="25%"/><img src="tmp_2211.02502/./f4.png" alt="Fig5.4" width="25%"/>

**Figure 5. -** Moment-0 (integrated intensity) images of the resolved sources in NaCl lines.
     For SrcI (top-left), this is the integrated intensity of the NaCl $J$=18-17 $v$=0
     line.  For the remainder, G17 (top right), G351 mm1 (bottom left), and NGC 6334I mm1b (bottom right),
     these are the average of the $J$=18-17 and $J$=17-16 transitions
     of both the $v$=0 and $v$=1 states.  The coordinates are given in RA/Dec offset from the
     central position specified under the abscissa.
     The scalebars show physical sizes as labeled.
     The ellipses in the corners show the full-width half-maximum beam ellipse. (*fig:resolveddisks*)

</div>
<div id="div_fig2">

<img src="tmp_2211.02502/./f29.png" alt="Fig13.1" width="25%"/><img src="tmp_2211.02502/./f30.png" alt="Fig13.2" width="25%"/><img src="tmp_2211.02502/./f31.png" alt="Fig13.3" width="25%"/><img src="tmp_2211.02502/./f32.png" alt="Fig13.4" width="25%"/>

**Figure 13. -** Stacked spectra from G351.77mm1 from the \citet{Beuther2019} data set.
    The stacking was based on the H$_2$O line.
    Line IDs are shown; no KCl detections are clear.
    Different colors are used for targeted species with multiple transitions in-band: orange for SiS, blue for KCl, red for NaCl, {magenta for H30$\alpha$, purple for PN, and green for $\water$}.
    The remaining species, with only one transition marked, are shown in black.
    The PN line identification should be taken with a grain of salt
    since it is the only transition we observe from PN.
     (*fig:g351mm1*)

</div>
<div id="div_fig3">

<img src="tmp_2211.02502/./f24.png" alt="Fig11" width="100%"/>

**Figure 11. -** Moment-0 (integrated intensity) and moment-1 (intensity-weighted velocity) images of {stacked} NaCl (left) and $\water$(right) for G351 mm1.
    The position-velocity diagrams in Figure \ref{fig:g351momentpv} are
    taken along position angle $130^\circ$, parallel to the velocity gradient seen in the lower panels here.
    {The red arrow shows the direction of the outflow from \citet{Beuther2019}{that extends $\gtrsim0.2$ pc to the northwest}.
    A compact blueshifted {feature} was also weakly detected b{in SiO} opposite the redshifted flow, but it is unresolved.
    {The blue side of the NaCl, $\water$, and (in Appendix \ref{appendix:g351}) SiS and PN corresponds with the SiO blueshifted lobe seen in Fig. 7 of \citet{Beuther2019}, but since its extent is limited to the $\lesssim100$ AU scale shown in this figure, we cannot confirm whether it is comprised of outflowing material.
    }} (*fig:g351brinemoments*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

82  publications in the last 7 days.
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers